<a href="https://colab.research.google.com/github/shashwat12jha/Exploratory-Project/blob/main/varanasi_tracking_final_mine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.4/881.4 kB 52.5 MB/s eta 0:00:00


In [ ]:
!pip install boxmot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 45.1 MB/s eta 0:00:00
   ━━━━━━━

In [ ]:
from ultralytics import YOLO
import os
os.environ['WANDB_DISABLED'] = 'true'
model = YOLO('/content/bestnew.pt')

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
import cv2
import numpy as np
from pathlib import Path

In [ ]:
  def create_video_writer(video_cap, output_filename):
    # Grab the width, height, and fps of the frames in the video stream.
    frame_width = int(video_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(video_cap.get(cv2.CAP_PROP_FPS))

    # Initialize the FourCC and a video writer object
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    writer = cv2.VideoWriter(output_filename, fourcc, fps, (frame_width, frame_height))

    return writer

In [ ]:
# pip install --force-reinstall boxmot
import torch

In [ ]:
from boxmot import BotSort
from boxmot.trackers.botsort.botsort_track import STrack
from collections import deque

# Initialize the tracker
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
tracker = BotSort(
    reid_weights = Path('osnet_x0_25_msmt17.pt'),  # which ReID model to use
    device = device,
    half = False
)

# Initialize YOLO model
yolo_model=YOLO('/content/bestnew.pt')

# Open the input video
input_video_path = '/content/videoplayback.mp4'
vid = cv2.VideoCapture(input_video_path)

if not vid.isOpened():
    print("Error: Could not open input video.")
    exit()

# Define the output video path
output_video_path = '/content/videoplayback.mp4'

# Create the video writer
out = create_video_writer(vid, output_video_path)
frame_count = 1000
counter=0
trail_path = {0:{},1:{},2:{},3:{},4:{},5:{},6:{},7:{},8:{},9:{},10:{},'fc':[]}
counter_arr = {0:{},1:{},2:{},3:{},4:{},5:{},6:{},7:{},8:{},9:{},10:{}}

while frame_count:
    counter += 1
    frame_count -= 1
    ret, im = vid.read()
    if not ret:
        trail_path['fc'].append(1000-frame_count)
        break

    try:
        # Run the YOLO model on the frame
        results = yolo_model(im)

        # Convert the detections to the required format: N X (x, y, x, y, conf, cls)
        dets = []
        for result in results:
            for detection in result.boxes.data.cpu().numpy():
                x1, y1, x2, y2, conf, cls = detection
                dets.append([x1, y1, x2, y2, conf, int(cls)])
        dets = np.array(dets)

        # Update tracker with detections
        tracker.update(dets, im)
        for a in tracker.active_tracks:
          if a.history_observations.maxlen==50:
            a.history_observations = deque(a.history_observations, maxlen=1000)

        tracker.plot_results(im, show_trajectories = True)

        # trail_hist
        for a in tracker.active_tracks:
          if len(a.history_observations)>2:
            trail_path[a.cls][a.id] = a.history_observations
            counter_arr[a.cls][a.id].append(counter)



        # Write the frame to the output video
        out.write(im)

    except Exception as e:
        print(f"An error occurred: {e}")
        break

vid.release()
out.release()
cv2.destroyAllWindows()

print(f"Tracking video saved to {output_video_path}")

/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
2024-10-02 15:38:09.781 | INFO     | boxmot.utils.torch_utils:select_device:52 - Yolo Tracking v11.0.2 🚀 Python-3.10.12 torch-2.2.2+cu121
CUDA:0 (Tesla T4, 15102MiB)
Downloading...
From: https://drive.google.com/uc?id=1sSwXSUlj4_tHZequ_iZ8w_Jh0VaRQMqF
To: /content/osnet_x0_25_msmt17.pt
100%|██████████| 3.06M/3.06M [00:00<00:00, 60.3MB/s]
2024-10-02 15:38:14.904 | SUCCESS  | boxmot.appearance.reid_model_factory:load_pretrained_weights:183 - Loaded pretrained weights from osnet_x0_25_msmt17.pt



0: 384x640 (no detections), 110.2ms
Speed: 6.7ms preprocess, 110.2ms inference, 20.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 25.1ms
Speed: 2.9ms preprocess, 25.1ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)


2024-10-02 15:38:17.220 | WARNING  | boxmot.motion.cmc.ecc:apply:93 - Affine matrix could not be generated: OpenCV(4.10.0) /io/opencv/modules/video/src/ecc.cpp:574: error: (-7:Iterations do not converge) NaN encountered. in function 'findTransformECC'
. Returning identity



0: 384x640 (no detections), 25.2ms
Speed: 3.2ms preprocess, 25.2ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)


2024-10-02 15:38:17.259 | WARNING  | boxmot.motion.cmc.ecc:apply:93 - Affine matrix could not be generated: OpenCV(4.10.0) /io/opencv/modules/video/src/ecc.cpp:574: error: (-7:Iterations do not converge) NaN encountered. in function 'findTransformECC'
. Returning identity


...


0: 384x640 (no detections), 11.7ms
Speed: 1.4ms preprocess, 11.7ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)


2024-10-02 15:38:17.878 | WARNING  | boxmot.motion.cmc.ecc:apply:93 - Affine matrix could not be generated: OpenCV(4.10.0) /io/opencv/modules/video/src/ecc.cpp:574: error: (-7:Iterations do not converge) NaN encountered. in function 'findTransformECC'
. Returning identity



0: 384x640 (no detections), 11.9ms
Speed: 1.3ms preprocess, 11.9ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)


2024-10-02 15:38:17.903 | WARNING  | boxmot.motion.cmc.ecc:apply:93 - Affine matrix could not be generated: OpenCV(4.10.0) /io/opencv/modules/video/src/ecc.cpp:574: error: (-7:Iterations do not converge) NaN encountered. in function 'findTransformECC'
. Returning identity


Tracking video saved to /content/videoplayback.mp4


In [ ]:
import copy
def path(trail_path):
  fc = trail_path.pop('fc')
  trail_path_cent_x = copy.deepcopy(trail_path)
  trail_path_cent_y = copy.deepcopy(trail_path)
  for cls in trail_path.keys():
    if cls == 'fc':
      continue
    for k in trail_path[cls].keys():
      for i in range(len(trail_path[cls][k])):
        # print(cls,k,i)
        # print(trail_path[cls][k][i])
        trail_path_cent_x[cls][k][i] = (trail_path[cls][k][i][0] + trail_path[cls][k][i][2])/2
        trail_path_cent_y[cls][k][i] = (trail_path[cls][k][i][1] + trail_path[cls][k][i][3])/2

  return trail_path_cent_x, trail_path_cent_y, fc

In [ ]:
trail_path_2 = copy.deepcopy(trail_path)
x, y, fc = path(trail_path_2)


In [ ]:
# for x-cordinates
import matplotlib.pyplot as plt

# figure and axes for the plots
fig, axes = plt.subplots(10, 1, figsize=(10, 40))

# Iterate over each class
for cls in range(10):
  ax = axes[cls]
  if cls in x.keys():
    # Iterate over each object ID for the current class
    for object_id in x[cls].keys():
      # Plot the x-coordinates for the current object ID
      ax.plot(range(len(x[cls][object_id])), x[cls][object_id], label=f'ID {object_id}')

    # Set plot title and labels
    ax.set_title(f'Class {cls} - X-Coordinates')
    ax.set_xlabel('Frame')
    ax.set_ylabel('X-Coordinate')
    ax.legend()

# Adjust layout and show the plots
plt.tight_layout()
plt.show()


In [ ]:
# for y-coordinates

import matplotlib.pyplot as plt

# Create a figure and axes for the plots
fig, axes = plt.subplots(10, 1, figsize=(10, 40))

# Iterate over each class
for cls in range(10):
  ax = axes[cls]
  if cls in y.keys():
    # Iterate over each object ID for the current class
    for object_id in y[cls].keys():
      # Plot the x-coordinates for the current object ID
      ax.plot(range(len(y[cls][object_id])), y[cls][object_id], label=f'ID {object_id}')

    # Set plot title and labels
    ax.set_title(f'Class {cls} - y-Coordinates')
    ax.set_xlabel('Frame')
    ax.set_ylabel('y-Coordinate')
    ax.legend()

# Adjust layout and show the plots
plt.tight_layout()
plt.show()

In [ ]:
#WRITING OUT TO A FILE

In [ ]:
# Assuming x and y are dictionaries where keys are classes (0-9) and values are dictionaries of object IDs
for cls in range(10):
    # Define a unique file name for each class
    output_filename = f'output_class_{cls}.txt'

    # Ensure the file is created in append mode if it doesn't exist
    with open(output_filename, 'a') as file:
        pass  # Ensure the file is created

    # If class exists in x and y dictionaries
    if cls in x.keys() and cls in y.keys():
        # Iterate over each object ID for the current class in x and y
         for object_id in x[cls].keys():
             if object_id in y[cls].keys():

                # Open the unique file for the current class and write x, y, and object ID
                with open(output_filename, 'a') as file:
                    file.write(f"{x[cls][object_id]} {y[cls][object_id]} {object_id} {(counter_arr[cls][object_id])/()}\n")

